## This Script is used to get the unique sequences from GraphAligner and measure the edit distance between the sequences from Bandage and GraphAligner

In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict
import gfapy

In [2]:
# Open the GFA file
file_path = "Real.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [ ]:
data_graph = []

In [ ]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

In [ ]:
df_graph['Sequence']

In [ ]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [ ]:
df_graph

In [ ]:
df_GraphAligner_Paths=pd.read_csv('Real_Results/SequenceBandageSPAlignerGraphAlignerReal.tsv', sep='\t')

In [ ]:
df_GraphAligner_Paths

In [ ]:
selected_rows_BandageVSGraphAligner = df_GraphAligner_Paths[df_GraphAligner_Paths['FinalResultBandageVSGraphAligner'] != 'Full']
selected_rows_BandageVSGraphAligner.columns

In [ ]:
df_BandageVSGraphAligner=selected_rows_BandageVSGraphAligner[['Query', 'Sequence', 'Sequence_Bandage',
       'Start_GraphAligner', 'End_GraphAligner', 'Path_GraphAligner',
       'FinalResultBandageVSGraphAligner']].copy()


In [ ]:
df_BandageVSGraphAligner.reset_index()

In [ ]:
df_BandageVSGraphAligner.to_csv('Real_Results/BandageVSGraphAlignerSequences.tsv',sep='\t')

In [ ]:
df_GraphAligner_Path= pd.read_csv('Real_Results/BandageVSGraphAlignerSequences.tsv', sep='\t')

In [ ]:
df_GraphAligner_Path

In [ ]:
df_GraphAligner_Path[['Path1', 'Path2','Path3','Path4','Path5','Path6','Path7','Path8','Path9','Path10','Path11']] = df_GraphAligner_Path['Path_GraphAligner'].str.split(',', expand=True)

In [ ]:
df_GraphAligner_Path

## Get the rows that have a unique node, Path1

In [ ]:
# Get the rows where Path 2 is  None
Path1_Only = df_GraphAligner_Path[df_GraphAligner_Path['Path2'].isna()]

# Print the resulting dataframe
Path1_Only 

In [ ]:
#Drop the 0 parts
Path1_Only = Path1_Only.drop(Path1_Only[(Path1_Only['Start_GraphAligner'] == 0) & (Path1_Only['End_GraphAligner'] == 0)].index)


In [ ]:
# Get the sequence where we have only Path1
merged_df_Path1= pd.merge(Path1_Only , df_graph, left_on='Path1', right_on='Name',how='inner')
merged_df_Path1

In [ ]:
merged_df_Path1.dtypes

In [ ]:
merged_df_Path1['Start_GraphAligner']=merged_df_Path1['Start_GraphAligner'].astype(int)
merged_df_Path1['End_GraphAligner']=merged_df_Path1['End_GraphAligner'].astype(int)

In [ ]:
merged_df_Path1.dtypes

In [ ]:
merged_df_Path1

In [ ]:
def extract_sequences_Path1(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start_GraphAligner']
        end_pos = row['End_GraphAligner']
        sequence = row['Sequence_y']
        
        extracted_sequence = sequence[start_pos-1:end_pos]
        dataframe.loc[index, 'Result_node1'] = extracted_sequence
    
    return dataframe

In [ ]:
extracted_sequences_Path1_df = extract_sequences_Path1(merged_df_Path1)
extracted_sequences_Path1_df
#Save the dataframe to a pdf
extracted_sequences_Path1_df.to_csv("111_GraphResults/Path1_TestSequences.tsv", sep="\t")

In [ ]:
extracted_sequences_Path1_df

In [ ]:
import Levenshtein
#Create a method to calculate the levenstein distance
def calculate_edit_distance_sequence_GraphAligner(row):
    return Levenshtein.distance(row['Sequence_x'], row['Result_node1'])

In [ ]:
extracted_sequences_Path1_df['edit_distance_query_sequence_GraphAligner'] = extracted_sequences_Path1_df.apply(calculate_edit_distance_sequence_GraphAligner, axis=1)

In [ ]:
extracted_sequences_Path1_df

In [ ]:
extracted_sequences_Path1_df['sequence_length'] = extracted_sequences_Path1_df['Sequence_x'].apply(len)
extracted_sequences_Path1_df['EditDistance_GraphAligner_Query']=1-(extracted_sequences_Path1_df['edit_distance_query_sequence_GraphAligner']/extracted_sequences_Path1_df['sequence_length'])

In [ ]:
extracted_sequences_Path1_df

In [ ]:
extracted_sequences_Path1_df.to_csv('111_GraphResults/GraphAlignerSequencesEditDistancePath1.tsv',sep='\t')

In [ ]:
# Get the rows where Path 2 is not None
df_multiple = df_GraphAligner_Path[df_GraphAligner_Path['Path2'].notna() ]

# Print the resulting dataframe
df_multiple

In [ ]:
merged_df_multiple_Path1= pd.merge(df_multiple, df_graph, left_on='Path1', right_on='Name')
merged_df_multiple_Path1


In [ ]:
merged_df_multiple_Path1['Path2'] = merged_df_multiple_Path1['Path2'].str.strip()


In [ ]:
merged_df_multiple_Path2=pd.merge(merged_df_multiple_Path1, df_graph,left_on='Path2', right_on='Name')
merged_df_multiple_Path2

In [ ]:
#Rename Name_X, Sequence_X, Name_y, Sequence_y
merged_df_multiple_Path2.rename(columns = {'Sequence':'Sequence_Path1'}, inplace = True)
merged_df_multiple_Path2.rename(columns = {'Sequence_y':'Sequence_Path2'}, inplace = True)
merged_df_multiple_Path2

In [ ]:
merged_df_multiple_Path2.dtypes

In [ ]:
merged_df_multiple_Path2

In [ ]:
merged_df_multiple_Path2.to_csv('merged_df_multiple_Path2.tsv',sep='\t')

In [ ]:
def extract_sequences_multiple_sequences(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start_GraphAligner']
        sequence_1 = row['Sequence_Path2']
       
        #Get the first sequence, and remove 55 kmer from it
        extracted_sequence_first = sequence_1[start_pos-1:]
        
        dataframe.loc[index, 'Sequencefirst'] = extracted_sequence_first
    
    return dataframe

In [ ]:
merged_df_multiple_Path2_modified=pd.read_csv('merged_df_multiple_Path2.tsv', sep='\t')

In [ ]:
merged_df_multiple_Path2_modified

In [ ]:
merged_df_multiple_Path2_modified['Start_GraphAligner']=merged_df_multiple_Path2['Start_GraphAligner'].astype(int)
merged_df_multiple_Path2_modified['End_GraphAligner']=merged_df_multiple_Path2['End_GraphAligner'].astype(int)

In [ ]:
extracted_sequences_2 = extract_sequences_multiple_sequences(merged_df_multiple_Path2)
extracted_sequences_2.to_csv('TestSeque.tsv',sep='\t')

extracted_sequences_2

In [ ]:
def extract_sequences_multiple_sequences_path2(dataframe):
    for index, row in dataframe.iterrows():
        end_pos = row['End_GraphAligner']
        sequence_2 = row['Sequence_Path1']
       
        #Get the first sequence, and remove 55 kmer from it
        extracted_sequence_second = sequence_2[:end_pos]
        
        dataframe.loc[index, 'SequencefSecond'] = extracted_sequence_second
    
    return dataframe

In [ ]:
extracted_sequences_3 = extract_sequences_multiple_sequences_path2(extracted_sequences_2)
extracted_sequences_3.to_csv('TestSeque3.tsv',sep='\t')

extracted_sequences_3

In [ ]:
def extract_sequences_multiple_sequences_overlap(dataframe):
   
    for index, row in dataframe.iterrows():
        sequence_first = row['Sequencefirst']
        length_end=len(sequence_first)-55
       
        #Get the first sequence, and the last 55 characters
        extracted_sequence_first = sequence_first[:len(sequence_first)-55]
        
        dataframe.loc[index, 'length'] = len(sequence_first)
        dataframe.loc[index, 'length_end'] = length_end
        dataframe.loc[index, 'FinalSequence1'] = sequence_first[:length_end]
    
    return dataframe
    

In [ ]:
extracted_sequences_multiple = extract_sequences_multiple_sequences_overlap(extracted_sequences_3)

In [ ]:
extracted_sequences_multiple

In [ ]:
def extract_sequences_multiple_sequences_final(dataframe):
   
    for index, row in dataframe.iterrows():
        sequence_first = row['FinalSequence1']
        sequence_second = row['SequencefSecond']
       
        #Get the first sequence, and the last 55 characters
        extracted_sequence_final = sequence_first+sequence_second
        
      
        dataframe.loc[index, 'FinalSequence'] = sequence_first + sequence_second
    
    return dataframe

In [ ]:
extracted_sequences_multiple_final =extract_sequences_multiple_sequences_final(extracted_sequences_multiple)

In [ ]:
extracted_sequences_multiple_final

In [ ]:
import Levenshtein
#Create a method to calculate the levenstein distance
def calculate_edit_distance_sequence_GraphAligner_multiple(row):
    return Levenshtein.distance(row['Sequence_x'], row['FinalSequence'])

In [ ]:
extracted_sequences_multiple_final['edit_distance_query_sequence_GraphAligner'] =calculate_edit_distance_sequence_GraphAligner_multiple(extracted_sequences_multiple_final)

In [ ]:
extracted_sequences_multiple_final

In [ ]:
extracted_sequences_multiple_final['sequence_length'] = extracted_sequences_multiple_final['Sequence_x'].apply(len)
extracted_sequences_multiple_final['EditDistance_GraphAligner_Query']=1-(extracted_sequences_multiple_final['edit_distance_query_sequence_GraphAligner']/extracted_sequences_multiple_final['sequence_length'])

In [ ]:
#Drop some columns
extracted_sequences_multiple_final=extracted_sequences_multiple_final.drop(['Path_Bandage', 'Path_GraphAligner','Equal','Name_x','Name_y'], axis=1)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequence_Path1':'Sequence_Path1'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequence_Path2':'Sequence_FromGraph_Path2'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequencefirst':'Sequence_FromGraph_Path1_No_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'length':'Length_Sequence_Path1_No_Overlap'}, inplace = True)


In [ ]:
extracted_sequences_multiple_final.rename(columns = {'length_end':'Length_Sequence_Path1_With_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'FinalSequence1':'Sequence_Path1_With_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'FinalSequence':'Concatenated_Sequence'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.to_csv("111_GraphResults/MultiplePath_TestSequences.tsv",sep='\t')

## This section measures the edit distance between the sequences 

In [ ]:
import Levenshtein

In [ ]:
bandage_df= pd.read_csv('Real_Results/TestDistance.tsv', sep='\t')
bandage_df

In [ ]:
#Get the sequences from bandage where the Path2 in GraphAligner=none
distance_df=pd.merge(extracted_sequences_Path1_df,bandage_df, on='Query')

In [ ]:
distance_df

In [ ]:
distance_df.rename(columns = {'Result_node1':'Sequence_GraphAligner'}, inplace = True)
distance_df.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)

In [ ]:
distance_df=distance_df.drop('Unnamed: 0_x', axis=1)

In [ ]:
distance_df=distance_df.drop('Unnamed: 0_y', axis=1)

In [ ]:
distance_df

In [ ]:
#Create a method to calculate the levenstein distance
def calculate_edit_distance_Bandage_GraphAligner(row):
    return Levenshtein.distance(row['Sequence_Bandage'], row['Sequence_GraphAligner'])

In [ ]:
distance_df['edit_distance_query_Bandage_GraphAligner'] = distance_df.apply(calculate_edit_distance_Bandage_GraphAligner, axis=1)

In [ ]:
distance_df.dtypes

In [ ]:
type_counts_bandage_GraphAligner=distance_df['edit_distance_query_Bandage_GraphAligner'].value_counts().reset_index()

In [ ]:
# Rename the columns
type_counts_bandage_GraphAligner.columns = ['Type', 'Count']

# Display the result
print(type_counts_bandage_GraphAligner)

In [ ]:
type_counts_bandage_GraphAligner.to_csv("Real_Results/type_counts_bandage_GraphAligner.tsv",sep='\t')

In [ ]:
#Get the sequences from bandage where the Path2 in GraphAligner is not none
distance_df_multiple=pd.merge(extracted_sequences_multiple_final,bandage_df, on='Query')
distance_df_multiple

In [ ]:
distance_df_multiple.rename(columns = {'Concatenated_Sequence':'Sequence_GraphAligner'}, inplace = True)
distance_df_multiple.rename(columns = {'Sequence':'Sequence_Bandage'}, inplace = True)

In [ ]:
distance_df_multiple['edit_distance_query_Bandage_GraphAligner'] = distance_df_multiple.apply(calculate_edit_distance_Bandage_GraphAligner, axis=1)

In [ ]:
distance_df_multiple

In [ ]:
type_counts_bandage_GraphAligner_multiple=distance_df_multiple['edit_distance_query_Bandage_GraphAligner'].value_counts().reset_index()

In [ ]:
# Rename the columns
type_counts_bandage_GraphAligner_multiple.columns = ['Type', 'Count']

# Display the result
print(type_counts_bandage_GraphAligner_multiple)

In [ ]:
type_counts_bandage_GraphAligner_multiple.to_csv("Real_Results/type_counts_bandage_GraphAligner_multiple.tsv",sep='\t')

In [ ]:
#Read the query and sequences from the .fasta file
fasta_file = "111_Graph/combined_1.fasta"  

# Create empty lists to store the data
queries = []
sequences = []

# Read through the FASTA file and extract query and sequence information
for record in SeqIO.parse(fasta_file, "fasta"):
    queries.append(record.id)
    sequences.append(str(record.seq))

# Create DataFrame from the lists
fasta_df = pd.DataFrame({'Query': queries, 'Sequence': sequences})

# Display the DataFrame
print(fasta_df)

In [ ]:
distance_df_Sequence=pd.merge(extracted_sequences_Path1_df,fasta_df, on='Query')

In [ ]:
distance_df_Sequence

In [ ]:
distance_df_Sequence.rename(columns = {'Result_node1':'Sequence_GraphAligner'}, inplace = True)
distance_df_Sequence.rename(columns = {'Sequence_y':'Sequence'}, inplace = True)

In [ ]:
distance_df_Sequence

In [ ]:
#Create a method to calculate the levenstein distance
def calculate_edit_distance_sequence_GraphAligner(row):
    return Levenshtein.distance(row['Sequence'], row['Sequence_GraphAligner'])

In [ ]:
distance_df_Sequence['edit_distance_query_sequence_GraphAligner'] = distance_df_Sequence.apply(calculate_edit_distance_sequence_GraphAligner, axis=1)

In [ ]:
distance_df_Sequence